<a href="https://colab.research.google.com/github/Elvenisboy/fuling_colaboratory/blob/master/AlexNet%E7%8C%AB%E7%8B%97%E5%A4%A7%E6%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# !cp /content/drive/colab-drive/cats-vs-dogs/dogs-vs-cats.zip ./
# !unzip ./dogs-vs-cats.zip
# !unzip ./test1.zip
!cp /content/drive/colab-drive/cats-vs-dogs/train.zip ./
!unzip ./train.zip

In [0]:
import os, shutil, glob

import cv2
import numpy as np

import keras
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions


In [0]:
def load_data():
  images = os.listdir('./train/')
  num = len(images)
  train_data = []
  train_label = []
  test_data = []
  test_label = []
#   train_data = np.empty((5000, 224, 224, 3), dtype="float32")
#   train_label = np.empty((5000, ), dtype="float32")
#   test_data = np.empty((5000, 224, 224, 3), dtype="float32")
#   test_label = np.empty((5000, ), dtype="float32")
  
  for i in range(5000):
    if i % 2:
      img = image.load_img('./train/' + 'dog.' + str(i) + '.jpg', target_size=(224, 224))
      x = image.img_to_array(img)
#       train_data[i] = x
      train_data.append(x)
      train_label.append(1)
    else:
      img = image.load_img('./train/' + 'cat.' + str(i) + '.jpg', target_size=(224, 224))
      x = image.img_to_array(img)
      train_data.append(x)
      train_label.append(0)
  for i in range(5000, 10000):
    if i % 2:
      img = image.load_img('./train/' + 'dog.' + str(i) + '.jpg', target_size=(224, 224))
      x = image.img_to_array(img)
      test_data.append(x)
      test_label.append(1)
    else:
      img = image.load_img('./train/' + 'cat.' + str(i) + '.jpg', target_size=(224, 224))
      x = image.img_to_array(img)
      test_data.append(x)
      test_label.append(0)
  return train_data, train_label, test_data, test_label        

In [0]:
train_data, train_label, test_data, test_label = load_data()
train_data, test_data = np.array(train_data), np.array(test_data)
# train_data, test_data = train_data/255, test_data/255


In [0]:
import matplotlib.pyplot as plt

img = train_data[0]
# print(img)
plt.imshow(img)
plt.axis("off")
plt.show()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis("off")
plt.show()


In [0]:
# keras.utils.to_categorical(y, num_classes=None, dtype='float32')
# num_classes: 总类别数。
train_label = keras.utils.to_categorical(train_label, 2)
test_label = keras.utils.to_categorical(test_label, 2)


In [0]:
# AlexNet
model = Sequential()
#第一段
# 3是应为图是彩色的
model.add(Convolution2D(filters=96, kernel_size=(11,11),
                 strides=(4,4), padding='valid',
                 input_shape=(224,224,3),
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), 
                       strides=(2,2), 
                       padding='valid'))
#第二段
model.add(Convolution2D(filters=256, kernel_size=(5,5), 
                 strides=(1,1), padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), 
                       strides=(2,2), 
                       padding='valid'))
#第三段
model.add(Convolution2D(filters=384, kernel_size=(3,3), 
                 strides=(1,1), padding='same', 
                 activation='relu'))
model.add(Convolution2D(filters=384, kernel_size=(3,3), 
                 strides=(1,1), padding='same', 
                 activation='relu'))
model.add(Convolution2D(filters=256, kernel_size=(3,3), 
                 strides=(1,1), padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), 
                       strides=(2,2), padding='valid'))
#第四段
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [0]:
model.fit(train_data, train_label,
          batch_size = 64,
          epochs = 50,
          validation_split = 0.2,
          shuffle = True)

# model.save('./outputs/cat_vs_dogs.h5')
# model.save('/content/drive/colab-drive/cats-vs-dogs/cat_vs_dogs_alex.h5')

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 17s 4ms/step - loss: 0.8608 - acc: 0.5097 - val_loss: 0.6758 - val_acc: 0.6140
Epoch 2/50
4000/4000 [==============================] - 11s 3ms/step - loss: 0.7031 - acc: 0.5363 - val_loss: 0.6732 - val_acc: 0.5610
Epoch 3/50
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6791 - acc: 0.5720 - val_loss: 0.6483 - val_acc: 0.6480
Epoch 4/50
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6606 - acc: 0.6085 - val_loss: 0.6749 - val_acc: 0.5720
Epoch 5/50
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6525 - acc: 0.6275 - val_loss: 0.6589 - val_acc: 0.5830
Epoch 6/50
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6297 - acc: 0.6508 - val_loss: 0.6671 - val_acc: 0.5960
Epoch 7/50
4000/4000 [==============================] - 11s 3ms/step - loss: 0.6094 - acc: 0.6623 - val_loss: 0.7005 - val_acc: 0.5770
Epoch 8

In [0]:
!mkdir output

model.save('/content/drive/colab-drive/cats-vs-dogs/cat_vs_dogs_alex_最终版.h5')
model.save('cat_vs_dogs.h5')

In [0]:
scores = model.evaluate(train_data, train_label, verbose=1)
print(scores)

In [0]:
scores = model.evaluate(test_data, test_label, verbose=1)
print(scores)

5000/5000 [==============================] - 6s 1ms/step
[0.9381468136310578, 0.8168]


In [0]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions


for i in range(100):
  if i % 2:
      img_path = './train/' + 'dog.' + str(i) + '.jpg'
  else:
      img_path = './train/' + 'cat.' + str(i) + '.jpg'
      
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  prediction = model.predict(x)
  print(prediction)
  print(np.argmax(prediction))
  print(100*np.max(prediction))



[[0.98972195 0.0102781 ]]
0
98.9721953868866
[[8.9023217e-07 9.9999917e-01]]
1
99.99991655349731
[[9.9999189e-01 8.1622275e-06]]
0
99.99918937683105
[[5.037856e-04 9.994962e-01]]
1
99.94962215423584
[[9.9999988e-01 1.3662347e-07]]
0
99.99998807907104
[[4.1485087e-06 9.9999583e-01]]
1
99.99958276748657
[[9.9929607e-01 7.0386776e-04]]
0
99.92960691452026
[[4.592663e-04 9.995408e-01]]
1
99.95408058166504
[[9.9998403e-01 1.5951522e-05]]
0
99.99840259552002
[[2.3513545e-05 9.9997652e-01]]
1
99.99765157699585
[[9.9999797e-01 2.0351451e-06]]
0
99.99979734420776
[[4.0044913e-08 1.0000000e+00]]
1
100.0
[[9.9999857e-01 1.4611421e-06]]
0
99.99985694885254
[[3.7508979e-04 9.9962485e-01]]
1
99.96248483657837
[[9.9999535e-01 4.6171708e-06]]
0
99.99953508377075
[[7.9438665e-05 9.9992061e-01]]
1
99.99206066131592
[[9.9998713e-01 1.2833714e-05]]
0
99.99871253967285
[[3.4434186e-05 9.9996555e-01]]
1
99.99655485153198
[[9.9990177e-01 9.8211065e-05]]
0
99.99017715454102
[[1.1416363e-04 9.9988580e-01]]
1
9

In [0]:
# 猫是0， 狗是1